# Code from my main EDA files that didn't quite work pasted here to keep things tidy

## Process

### Import

In [1]:
import pandas as pd
from datetime import datetime as dt
import time
import matplotlib.pyplot as plt
import numpy as np
from sys import getsizeof as ram
import networkx as nx
import squarify
from pyvis.network import Network

In [2]:
start = 0
def dtn():
    """Sets start variable to the current date and time
    
    IMPORTANT: must define global start variable before using"""
    global start
    start = dt.now()
    
def end():
    """Using in conjunction with dtn() function, returns the time taken between dtn() and this function being called"""
    return str(dt.now()-start)

def gen_end():
    """Generic function that gives a string telling how long it has been since dtn() was called. Can be used instead of end()"""
    stop = end()
    print("Completed in :"+stop)

In [165]:
try:
    print("Looking for local copy of data...")
    dtn()
    df_p = pd.read_csv("G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/Advanced-Cyber-Analytics-for-Attack-Detection/Data/Process data.gz",compression="gzip",index_col=0)
    stop = end()
    print("Data fetched locally in "+stop)
except OSError as e:
    if e.errno == 2:
        print("No Local data found. Importing from the web.")
        dtn()
        df_p = pd.read_csv("https://www.dropbox.com/s/c065rcq72abzm0s/Process_data.gz?dl=1",compression="gzip")
        stop = end()
        print("Data fetched from web in "+stop)
    else:
        print(e)

Looking for local copy of data...


G:\Programs\Anaconda\envs\det\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Data fetched locally in 0:01:07.118522


### Old NetX Graph Testing 1

In [232]:
G = nx.from_pandas_edgelist(df_un_top_10000.head(100),source="UserName",target="Device")
len(list(nx.connected_components(G)))

96

In [236]:
for component in list(nx.connected_components(G)):
    if len(component)<3:
        for node in component:
            G.remove_node(node)

In [237]:
len(list(nx.connected_components(G)))

1

### Old NetX Graph Testing 2 - experimentation for colouring

In [8]:
G = nx.from_pandas_edgelist(df_p.head(1000),source="ParentProcessName",target="ProcessName")

In [9]:
net = Network(notebook=True)
net.from_nx(G)

In [44]:
net.get_node()

{'size': 10, 'id': 'Proc391839.exe', 'label': 'Proc391839.exe', 'shape': 'dot'}

In [16]:
str(int(20.0))

'20'

In [55]:
sources = []
targets = []
for e in G.edges():
    source, target = e
    sources.append(source)
    targets.append(target)

In [66]:
listes = []
for i in G.nodes():
    listes.append(i)

In [68]:
print(len(listes))
print(len(sources))
print(len(targets))

298
94
244


In [59]:
sources = list(dict.fromkeys(sources))
targets = list(dict.fromkeys(targets))

In [41]:
if len(net.get_nodes()) == len(set(net.get_nodes())):
    print(False)
else:
    print(True)

False


In [24]:
df1 = df_p.drop_duplicates(subset="ProcessName")["ProcessName"]
df1

0           Proc391839.exe
1             rundll32.exe
2           Proc417435.exe
3           Proc249569.exe
4           Proc402696.exe
                 ...      
55677241    Proc260059.exe
55712526    Proc206206.exe
55729823    Proc371180.exe
55808235    Proc878594.exe
55870797    Proc280856.exe
Name: ProcessName, Length: 25373, dtype: object

In [23]:
df2 = df_p.drop_duplicates(subset="ParentProcessName")["ParentProcessName"]
df2

0           Proc387473
1             services
2           Proc417435
6           Proc721583
8              svchost
               ...    
55685539    Proc055438
55702364    Proc981496
55712672    Proc598940
55720101    Proc371180
55726326    Proc450659
Name: ParentProcessName, Length: 12226, dtype: object

In [27]:
df3 = df1.isin(df2)
o = 0
for i in df3:
    if i == True:
        o=o+1
print(o)

1173


In [36]:
df2[220:221]

7609    Proc391839.exe
Name: ParentProcessName, dtype: object

###  Daily Count - ALSO KIND OF USELESS AT THE MOMENT

In [16]:
df_p_2 = df_p[["UserName","DailyCount"]]
df_p_2.head()

,UserName,DailyCount
0,Comp748297$,1
1,Comp563664$,1
2,User607396,1
3,Comp641702$,1
4,Comp157389$,1


In [14]:
df_p_2.loc[df_p_2['UserName'] == "Comp563664$"]

,UserName,DailyCount
1,Comp563664$,1
4805,Comp563664$,2
16699,Comp563664$,13
24240,Comp563664$,18
26433,Comp563664$,1
...,...,...
55944564,Comp563664$,1
55960676,Comp563664$,271
55960798,Comp563664$,22
55962355,Comp563664$,5


In [26]:
df_p_dup = df_p_2.drop_duplicates(subset=["UserName"])

In [77]:
df_p_dup["DailyCount"].values[:] = 0

,UserName,DailyCount
0,Comp748297$,0
1,Comp563664$,0
2,User607396,0
3,Comp641702$,0
4,Comp157389$,0


In [27]:
#df_p_3 = df_p_2.replace('None','').groupby('UserName',as_index=False).agg(.join)

In [29]:
df_p_dup.shape

(25320, 2)

In [78]:
df_k = df_p_dup
#for i in df_p_dup["UserName"]:
df_k.head()

,UserName,DailyCount
0,Comp748297$,0
1,Comp563664$,0
2,User607396,0
3,Comp641702$,0
4,Comp157389$,0


In [135]:
df_k = df_p_dup
pd.options.mode.chained_assignment = None  # default='warn'
dtn()
for i in df_p_dup["UserName"]:
    df_k.loc[df_k["UserName"] == i,"DailyCount"] = df_p.loc[df_p["UserName"] == i, "DailyCount"].sum()
gen_end()

KeyboardInterrupt: 

In [150]:
df_k.head(1500)

,UserName,DailyCount
0,Comp748297$,85121
1,Comp563664$,84779
2,User607396,4496
3,Comp641702$,90766
4,Comp157389$,107357
...,...,...
1632,Comp288097$,0
1633,User862331,0
1634,Comp000577$,0
1635,Comp989307$,0


In [64]:
df_p.loc[df_p["UserName"] == "User024853"]["DailyCount"].sum()

4

In [133]:
pd.options.mode.chained_assignment = None  # default='warn'
df_k.loc[df_k["UserName"] == "User024853","DailyCount"] = df_p.loc[df_p["UserName"] == "User024853", "DailyCount"].sum()

In [131]:
df_k.loc[df_k["UserName"] == "User024853"]

,UserName,DailyCount
28791135,User024853,4


<bound method Index.get_loc of Int64Index([28791135], dtype='int64')>

In [21]:
print(df_p_2.shape)
print(df_p_3.shape)

(55981618, 2)
(25320, 2)


In [23]:
df_p_3.loc[df_p_3['UserName'] == "Comp563664$"]

,UserName,DailyCount
7467,Comp563664$,UserNameDailyCount
